 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Cell-Formatting" data-toc-modified-id="Cell-Formatting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cell Formatting</a></span></li><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Data-Collection" data-toc-modified-id="Data-Collection-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Collection</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Variance-Filtering" data-toc-modified-id="Variance-Filtering-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Variance Filtering</a></span></li><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Weightening" data-toc-modified-id="Weightening-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Weightening</a></span></li><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Data-scalling" data-toc-modified-id="Data-scalling-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Data scalling</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Dendrogram-Formation" data-toc-modified-id="Dendrogram-Formation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dendrogram Formation</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#display-features-list-with-indexes" data-toc-modified-id="display-features-list-with-indexes-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>display features list with indexes</a></span></li><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#data-plot-per-cluster" data-toc-modified-id="data-plot-per-cluster-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>data plot per cluster</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/affinity_propagation_sklearn.ipynb#Clustering-Metrics" data-toc-modified-id="Clustering-Metrics-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Clustering Metrics</a></span></li></ul></div>

# Cell Formatting

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Parameters

In [ ]:
OUTPUT_DIR = './../datasets/polvo/full_scenario/'
DATASET_PATH = './../datasets/polvo/full_scenario/dataset.csv'
VARIANCE_FILTER = 0.1
SCENARIO = ''
PEREFERENCE = -50

In [ ]:
from os.path import join
VARIANCE_FILTER = round(float(VARIANCE_FILTER), 2)

# Data Collection

In [ ]:
import pandas as pd
#from IPython.display import display

dataset = pd.read_csv(DATASET_PATH, delimiter=',')

pd.options.display.max_columns = None
pd.options.display.max_rows = None

target_labels = dataset['petrofacie']
petrofacies = list(set(target_labels))
dataset.index = [
    str(i) + '-' + str(petrofacie)
    for i, petrofacie in enumerate(dataset['petrofacie'])
]
targets = list(dataset['petrofacie'])

del dataset['petrofacie']
del dataset['lamina:'] # Removing well ID

feature_names = list(dataset.columns.values)
# dataset.describe()

## Variance Filtering

In [ ]:
print('from ' + str(dataset.shape[1]))
dataset = dataset.loc[:,dataset.std()>VARIANCE_FILTER]
# dataset = dataset.loc[:,dataset.std()<2-0.1*VARIANCE_FILTER]
print('to ' + str(dataset.shape[1]) + ' features')
processed_features = dataset.shape[1]

## Weightening

In [ ]:
# dataset.loc[:, 'Main/single size mode(mm):'] *= 10

## Data scalling

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from copy import deepcopy

# scalled_dataset = deepcopy(dataset)
dataset[:] = StandardScaler().fit_transform(X=dataset)


# Dendrogram Formation

In [ ]:
from sklearn.cluster import AffinityPropagation

km = AffinityPropagation(preference=PREFERENCE).fit(dataset)
labels = km.labels_

## display features list with indexes

In [ ]:
# pd.options.display.max_colwidth = 180
# display(pd.DataFrame(dataset.columns.values, columns=['FEATURE']))

## data plot per cluster
Boxplot features grouped according to the samples clustered

# Clustering Metrics

In [ ]:
from os.path import join
from sklearn import metrics

save_dic = {
    'metric_adjusted_rand_score':  [metrics.cluster.adjusted_rand_score(targets, labels)],
    'metric_adjusted_mutual_info_score': [metrics.cluster.adjusted_mutual_info_score(targets, labels)],
    'metric_homogeneity_score': [metrics.cluster.homogeneity_score(targets, labels)],
    'metric_completeness_score':[metrics.cluster.completeness_score(targets, labels)],
    'metric_v_measure_score': [metrics.cluster.v_measure_score(targets, labels)],
    'processed_features': [processed_features]
}
save_df = pd.DataFrame.from_dict(save_dic)
save_df.to_csv(join(OUTPUT_DIR, 'output_variance'+str(VARIANCE_FILTER)+'_preference_'+str(PREFERENCE)+'.csv'))

In [ ]:
from os.path import join

clusters_df = [pd.DataFrame() for _, _ in enumerate(set(labels))]

for i, label in enumerate(labels):
    clusters_df[label] = clusters_df[label].append(dataset.iloc[i,:])

output_dir = join(OUTPUT_DIR, 'output_variance'+str(VARIANCE_FILTER)+'_preference_'+str(PREFERENCE)+'_clusters.xlsx')
writer = pd.ExcelWriter(output_dir)

for i, cluster_df in enumerate(clusters_df):
    cluster_df = cluster_df.append(cluster_df.describe())
    cluster_df.to_excel(writer, 'Cluster'+str(i))    
writer.save()